In [19]:
!pip install -q cassio datasets langchain openai tiktoken langchain_community langchain_groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.6/129.6 kB 10.2 MB/s eta 0:00:00


In [20]:
# LangChain components to use
from langchain.vectorstores.cassandra import Cassandra
from langchain.indexes.vectorstore import VectorStoreIndexWrapper
from langchain.llms import OpenAI
from langchain_groq import ChatGroq

# Support for dataset retrieval with Hugging Face
from datasets import load_dataset

# With CassIO, the engine powering the Astra DB integration in LangChain,
# you will also initialize the DB connection:
import cassio

In [4]:
!pip install PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 13.7 MB/s eta 0:00:00


In [5]:
from PyPDF2 import PdfReader

In [ ]:
ASTRA_DB_APPLICATION_TOKEN = "Add your own" # enter the "AstraCS:..." string found in in your Token JSON file
ASTRA_DB_ID =  "Add your own" # enter your Database ID

# OPENAI_API_KEY = "sk-J3ZbnEqytFesD7kWKuVaT3BlbkVr9dpDbViv2R2un" # enter your OpenAI key

In [7]:
# provide the path of  pdf file/files.
pdfreader = PdfReader('/content/coi-4March2016.pdf')

In [8]:
from typing_extensions import Concatenate
# read text from pdf
raw_text = ''
for i, page in enumerate(pdfreader.pages):
    content = page.extract_text()
    if content:
        raw_text += content

In [9]:
raw_text

'THE\nCONSTITUTION\nOF INDIATHE\nCONSTITUTION\nOF INDIA\n(As on 9th November, 2015)\nPublished by\nGOVERNMENT OF INDIA\nMINISTRY OF LAW AND JUSTICE\n(LEGISLATIVE DEPARTMENT)\nPublished by\nGOVERNMENT OF  INDIA\nMINISTRY OF  LAW AND  JUSTICE\n(LEGISLATIVE  DEPARTMENT )\nNEW DELHI - 110 001\nPrinted by\nJAINCO ART INDIA13/10, W.E.A., Karol Bagh,New Delhi-110 005.PREFACE\nConstitution is a living document, an instrument which makes the\ngovernment system work. Its flexibility lies in its amendments. In this edition,\nthe text of the Constitution of India has been brought up-to-date byincorporating therein all amendments made by Parliament up to andincluding the Constitution (One Hundredth Amendment) Act, 2015 whichcontains details of acquired and transferred territories between theGovernments of India and Bangladesh and the same has been included inAnnexure.\nThe Constitution applies to the State of Jammu and Kashmir with certain\nexceptions and modifications as provided in article 370 an

In [21]:
llm =ChatGroq(model="gemma2-9b-it", groq_api_key='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')

In [10]:
cassio.init(token=ASTRA_DB_APPLICATION_TOKEN, database_id=ASTRA_DB_ID)

In [11]:
cassio.init(token=ASTRA_DB_APPLICATION_TOKEN, database_id=ASTRA_DB_ID)

In [12]:
from langchain_community.embeddings import HuggingFaceEmbeddings

embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


<ipython-input-12-436628643e16>:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or da

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [13]:
astra_vector_store = Cassandra(
    embedding=embedding,
    table_name="qa_mini_demo",
    session=None,
    keyspace=None,
)

In [14]:
from langchain.text_splitter import CharacterTextSplitter
# We need to split the text using Character Text Split such that it sshould not increse token size
text_splitter = CharacterTextSplitter(
    separator = "\n",
    chunk_size = 800,
    chunk_overlap  = 200,
    length_function = len,
)
texts = text_splitter.split_text(raw_text)

In [15]:
texts[:50]

['THE\nCONSTITUTION\nOF INDIATHE\nCONSTITUTION\nOF INDIA\n(As on 9th November, 2015)\nPublished by\nGOVERNMENT OF INDIA\nMINISTRY OF LAW AND JUSTICE\n(LEGISLATIVE DEPARTMENT)\nPublished by\nGOVERNMENT OF  INDIA\nMINISTRY OF  LAW AND  JUSTICE\n(LEGISLATIVE  DEPARTMENT )\nNEW DELHI - 110 001\nPrinted by\nJAINCO ART INDIA13/10, W.E.A., Karol Bagh,New Delhi-110 005.PREFACE\nConstitution is a living document, an instrument which makes the\ngovernment system work. Its flexibility lies in its amendments. In this edition,',
 'Constitution is a living document, an instrument which makes the\ngovernment system work. Its flexibility lies in its amendments. In this edition,\nthe text of the Constitution of India has been brought up-to-date byincorporating therein all amendments made by Parliament up to andincluding the Constitution (One Hundredth Amendment) Act, 2015 whichcontains details of acquired and transferred territories between theGovernments of India and Bangladesh and the same has been i

In [16]:
astra_vector_store.add_texts(texts[:50])

print("Inserted %i headlines." % len(texts[:50]))

astra_vector_index = VectorStoreIndexWrapper(vectorstore=astra_vector_store)

Inserted 50 headlines.


In [ ]:
first_question = True
while True:
    if first_question:
        query_text = input("\nEnter your question (or type 'quit' to exit): ").strip()
    else:
        query_text = input("\nWhat's your next question (or type 'quit' to exit): ").strip()

    if query_text.lower() == "quit":
        break

    if query_text == "":
        continue

    first_question = False

    print("\nQUESTION: \"%s\"" % query_text)
    answer = astra_vector_index.query(query_text, llm=llm).strip()
    print("ANSWER: \"%s\"\n" % answer)

    print("FIRST DOCUMENTS BY RELEVANCE:")
    for doc, score in astra_vector_store.similarity_search_with_score(query_text, k=4):
        print("    [%0.4f] \"%s ...\"" % (score, doc.page_content[:84]))


Enter your question (or type 'quit' to exit): does india offers right to freedom of religion?



QUESTION: "does india offers right to freedom of religion?"


ANSWER: "Yes, according to the provided text, India offers the right to freedom of religion.  

This is found under the section "Right to Freedom of Religion" which includes articles 25 through 28."

FIRST DOCUMENTS BY RELEVANCE:
    [0.7696] "21A. Right to education ............................................................ ..."
    [0.7689] "26. Freedom to manage religious affairs ............................................ ..."
    [0.7020] "6. Rights of citizenship of certain persons who have migrated to Indiafrom Pakistan  ..."
    [0.6864] "Constitution is a living document, an instrument which makes the
government system w ..."

What's your next question (or type 'quit' to exit): what is right to freedon of religion?



QUESTION: "what is right to freedon of religion?"


ANSWER: "The text provides information on the Right to Freedom of Religion. 

It states that this right includes:

* **Freedom of conscience and free profession, practice and propagation of religion:**  This means individuals are free to believe what they want, practice their religion openly, and share their beliefs with others.
* **Freedom to manage religious affairs:** This grants religious groups the autonomy to govern their own institutions and practices.
* **Freedom as to payment of taxes for the promotion of any particular religion:**  Individuals are not required to pay taxes that directly support a specific religion.
* **Freedom as to attendance at religious instruction or religious worship in certain educational institutions:**  This protects the right of students to opt out of religious instruction or worship in schools. 


The text lists these articles under the heading "Right to Freedom of Religion":

* 25. Freedom of conscience and free profession, practice and propagation


QUESTION: "what is Right against Exploitation"


ANSWER: "Right against Exploitation is covered in articles 23 and 24."

FIRST DOCUMENTS BY RELEVANCE:
    [0.6857] "21A. Right to education ............................................................ ..."
    [0.6811] "(i)ARTICLE PAGES
16. Equality of opportunity in matters of public employment ....... ..."
    [0.6558] "37. Application of the principles contained in this Part ........................... ..."
    [0.6348] "46. Promotion of educational and economic interests of Scheduled Castes,Scheduled Tr ..."
